In [9]:
#!/usr/bin/env python
# coding: utf-8

from gurobipy import *
import numpy as np
import pandas as pd 
import numpy as np
os.chdir("C:\\Users\\uqrbean1\\OneDrive - The University of Queensland\\Documents\\ieee")

flist = [    
    "phase2_instance_small_0.txt",
    "phase2_instance_small_1.txt",
    "phase2_instance_small_2.txt",
    "phase2_instance_small_3.txt",
    "phase2_instance_small_4.txt",
    "phase2_instance_large_0.txt",
    "phase2_instance_large_1.txt",
    "phase2_instance_large_2.txt",
    "phase2_instance_large_3.txt",
    "phase2_instance_large_4.txt"    
        ]

# slist = [ 
#     "flatsmall0y_0.sol",
#     "flatsmall1_29.sol",
#     "flatsmall2_23.sol",
#     "flatsmall3_24.sol",
#     "flatsmall4_0.sol",
#     "flatlarge0y_3.sol",
#     "flatlarge1y_3.sol",
#     "flatlarge2y_5.sol",
#     "flatlarge3y_4.sol",
#     "flatlarge4y_4.sol"
# ]

for nu in range(0,10):
    file = flist[nu]
#     sol = slist[nu]
    count_data = np.genfromtxt(file,delimiter=' ',max_rows=1,usecols=(1,2,3,4,5),dtype=int)
    building_count = count_data[0]
    solar_count = count_data[1]
    battery_count = count_data[2]
    recurring_count = count_data[3]
    once_count = count_data[4]
    building_data = np.genfromtxt(file,delimiter=' ',skip_header=1,max_rows=building_count,usecols=(1,2,3),dtype=int)
    building_num = building_data[:,0]
    building_small = building_data[:,1]
    building_large = building_data[:,2]
    battery_data = np.genfromtxt(file,delimiter=' ',skip_header=1+solar_count+building_count,max_rows=battery_count,
                                 usecols=(1,3,4,5),dtype=float)

    with open(file, 'r') as f:
        file_lines = [line.rstrip() for line in f]

    df =  pd.DataFrame([string.split(' ') for string in file_lines])
    alldata = np.array(df)

    recur_data = alldata[alldata[:,0]=='r',]
    once_data = alldata[alldata[:,0]=='a',]

    recur_small = recur_data[recur_data[:,3]=='S']
    recur_large = recur_data[recur_data[:,3]=='L']
    
    once_small = once_data[once_data[:,3]=='S']
    once_large = once_data[once_data[:,3]=='L']

    T               = 2880                                    # number of time periods
    timeperiods     = range(T)   
    
    T1 = 532
    timeperiods1 = range(T1)
   
    nBuildings = building_count 
    buildings = range(nBuildings)
    
    smallActIdx = pd.to_numeric(recur_small[:,1])
    largeActIdx = pd.to_numeric(recur_large[:,1])    
        
    smallOnceIdx = pd.to_numeric(once_small[:,1])
    largeOnceIdx = pd.to_numeric(once_large[:,1])

    smallAct = range(smallActIdx.shape[0])
    largeAct = range(largeActIdx.shape[0])
    
    smallOnce = range(smallOnceIdx.shape[0])
    largeOnce = range(largeOnceIdx.shape[0])

    ActEnergy = pd.to_numeric(recur_data[:,4])
    OnceEnergy = pd.to_numeric(once_data[:,4])

    smallActPeriods = recur_small[:,5]
    largeActPeriods = recur_large[:,5]
    ActPeriods = pd.to_numeric(recur_data[:,5])
    
    smallOncePeriods = once_small[:,5]
    largeOncePeriods = once_large[:,5]
    OncePeriods = pd.to_numeric(once_data[:,5])

    ActPrec = pd.to_numeric(recur_data[:,6])
    
    OncePeak = pd.to_numeric(once_data[:,6]) ## benefit if scheduled in peak
    OnceOffpeak = pd.to_numeric(once_data[:,7]) ## benefit if scheduled off peak
    OncePrec = pd.to_numeric(once_data[:,8])

    ActRooms = pd.to_numeric(recur_data[:,2])
    OnceRooms = pd.to_numeric(once_data[:,2])

    Act = range(recurring_count)
    Once = range(once_count)
       
    week1 = list(range(88,120)) + list(range(184,216)) + list(range(280,312)) + list(range(376,408)) + list(range(472,504))
    week2 = list(range(88+672,120+672)) + list(range(184+672,216+672)) + list(range(280+672,312+672)) + list(range(376+672,408+672)) + list(range(472+672,504+672))
    week3 = list(range(88+672*2,120+672*2)) + list(range(184+672*2,216+672*2)) + list(range(280+672*2,312+672*2)) + list(range(376+672*2,408+672*2)) + list(range(472+672*2,504+672*2))
    week4 = list(range(88+672*3,120+672*3)) + list(range(184+672*3,216+672*3)) + list(range(280+672*3,312+672*3)) + list(range(376+672*3,408+672*3)) + list(range(472+672*3,504+672*3))
    week5 = list(range(88+672*4,120+672*4)) + list(range(184+672*4,2880)) # Monday 30 Nov and 8 periods of Tuesday 1 Dec
   
    weeks = week1 + week2 + week3 + week4 + week5
    
    notweek = list(set(timeperiods) - set(weeks))
    notweek1 = list(set(timeperiods) - set(week1))
    
    m = Model("monash_schedule")

    w            = m.addVars(Act, timeperiods, lb=-1, ub=1, name="w") # intermediate variable for differences
    v            = m.addVars(Act, timeperiods, name="v") # max v 
    s            = m.addVars(Act, timeperiods, vtype=GRB.BINARY, name="s") # binary allocation variable

    bu           = m.addVars(Act, buildings, lb=0, ub=3, vtype=GRB.INTEGER, name="roomlimit") # small_ac_buildings - sum to activity room count
    small_use       = m.addVars(timeperiods, buildings, lb=0, name="small_use")
    large_use       = m.addVars(timeperiods, buildings, lb=0, name="large_use")

    wo            = m.addVars(Once, timeperiods, lb=-1, ub=1, name="wo") # intermediate variable for differences
    vo            = m.addVars(Once, timeperiods, name="vo") # max v 
    so            = m.addVars(Once, timeperiods, vtype=GRB.BINARY, name="so") # binary allocation variable

    buo           = m.addVars(Once, buildings, lb=0, ub=3, vtype=GRB.INTEGER, name="roomlimito")
    #buoo          = m.addVars(Once, buildings, lb=0, ub=3, vtype=GRB.INTEGER, name="roomlimitoo")
    
    onceLoad       = m.addVars(Once, timeperiods, name="once_load")   
    #onceIn = m.addVars(Once, name="oncein", vtype=GRB.BINARY)    
    
    actLoad      = m.addVars(Act, timeperiods, name="load")
    load         = m.addVars(timeperiods, name="load1") # a variable load -- add net load and activity load and batteries 
    maxrecuronceload      = m.addVar(name="maxrecuronceload")
    
    #obj_lin      = m.addVar(name="obj_lin")
    recurload = m.addVars(timeperiods1, name="recurload")
    onceload = m.addVars(timeperiods, name="once_load")
    
    z  = m.addVars(Act, lb=0,ub=T1-1, name="z")
    z0 = m.addVars(Act, timeperiods1, name="z0")
    zo  = m.addVars(Once, lb=0,ub=T-1, name="zo")
    dayzo  = m.addVars(Once, lb=0,ub=T-1, name="dayzo", vtype=GRB.INTEGER)    
    
    z0o= m.addVars(Once, timeperiods, name="z0o")    
    
    for a in Act:
        m.addConstrs((z0[(a,t)] == t*v[(a,t)]) for t in timeperiods1)

    for a in Act:
        m.addConstr(z[a] == max_(z0[a,t] for t in timeperiods1))
        
    for o in Once:
        m.addConstrs((z0o[(o,t)] == t*vo[(o,t)]) for t in timeperiods)

    for o in Once:
        m.addConstr(zo[o] == max_(z0o[o,t] for t in timeperiods))

    m.addConstrs(zo[o]+44 >= dayzo[o]*96 for o in Once)
    m.addConstrs(zo[o]+44 <= dayzo[o]*96 + 95 for o in Once)
        
    m.setObjective(maxrecuronceload, GRB.MINIMIZE)
    
    #constr = m.addConstrs((onceIn[o] == 1 for o in Once), name="onceall")
    #for o in Once:
    #    constr[o].Lazy = -1
        
    constr = m.addConstrs((w[(a,t)] == (s[(a,t)] - s[(a,t-1)]) for a in Act for t in range(1,T1)), name="shift_starts_hat")
    constr = m.addConstrs((v[(a,t)] == max_(w[(a,t)], 0.0) for a in Act for t in range(1,T1)), name="shift_starts")
    constr = m.addConstrs((v[(a,0)] == s[(a,0)] for a in Act), name="shift_starts_init")
    constr = m.addConstrs((quicksum([v[(a,t)] for t in timeperiods1]) <= 1 for a in Act), name="shift_start_max")
    
    constr = m.addConstrs((wo[(o,t)] == (so[(o,t)] - so[(o,t-1)]) for o in Once for t in range(1,T)), name="shift_starts_hato")
    constr = m.addConstrs((vo[(o,t)] == max_(wo[(o,t)], 0.0) for o in Once for t in range(1,T)), name="shift_startso")
    constr = m.addConstrs((vo[(o,0)] == so[(o,0)] for o in Once), name="shift_starts_inito")
    constr = m.addConstrs((quicksum([vo[(o,t)] for t in timeperiods]) <= 1 for o in Once), name="shift_start_maxo")
    
    constr = m.addConstrs((recurload[t] == (quicksum([actLoad[(a,t)] for a in Act])) for t in week1), name="netloada")
    constr = m.addConstrs((onceload[t] == (quicksum([onceLoad[(o,t)] for o in Once])) for t in weeks), name="netloadb")
    
    constr = m.addConstr(maxrecuronceload == max_([load[t] for t in weeks]),name="get_max_load1")
       
    #constr = m.addConstrs((load[t] == (  onceload[t] ) for t in range(0,88)), name="netload1")
    #constr = m.addConstrs((load[t] == (  onceload[t] ) for t in range(504,88+672)), name="netload3")
    #constr = m.addConstrs((load[t] == (  onceload[t] ) for t in range(504+672,88+672*2)), name="netload5")
    #constr = m.addConstrs((load[t] == (  onceload[t] ) for t in range(504+672*2,88+672*3)), name="netload7")
    #constr = m.addConstrs((load[t] == (  onceload[t] ) for t in range(504+672*3,88+672*4)), name="netload9")
    
    constr = m.addConstrs((load[t] == (onceload[t] + recurload[t]) for t in week1), name="netload2")
    constr = m.addConstrs((load[t] == (onceload[t] + recurload[t-672]) for t in week2), name="netload4")
    constr = m.addConstrs((load[t] == (onceload[t] + recurload[t-672*2]) for t in week3), name="netload6")
    constr = m.addConstrs((load[t] == (onceload[t] + recurload[t-672*3]) for t in week4), name="netload8")
    constr = m.addConstrs((load[t] == (onceload[t] + recurload[t-672*4]) for t in week5), name="netloadaa")
    
    #constr = m.addConstr((obj_lin == quicksum([(load[t] * price[t]) for t in timeperiods])/4000) , "linear_objective")
    
    constr = m.addConstrs((quicksum([s[(a,t)] for t in week1]) == ActPeriods[a] for a in Act), name="col")
    constr = m.addConstrs((quicksum([so[(o,t)] for t in weeks]) == OncePeriods[o] for o in Once), name="colo")

    for b in buildings:
        for t in week1:
            constr = m.addConstr((small_use[(t,b)] == quicksum([bu[(a,b)]*s[(a,t)] for a in smallActIdx]) + 
                                                      quicksum([buo[(o,b)]*so[(o,t)] for o in smallOnceIdx])),name="buso1")
            
    for b in buildings:
        for t in week2:
            constr = m.addConstr((small_use[(t,b)] == quicksum([bu[(a,b)]*s[(a,t-672)] for a in smallActIdx]) + 
                                                      quicksum([buo[(o,b)]*so[(o,t)] for o in smallOnceIdx])),name="buso2")

    for b in buildings:
        for t in week3:
            constr = m.addConstr((small_use[(t,b)] == quicksum([bu[(a,b)]*s[(a,t-672*2)] for a in smallActIdx]) + 
                                                      quicksum([buo[(o,b)]*so[(o,t)] for o in smallOnceIdx])),name="buso3")

    for b in buildings:
        for t in week4:
            constr = m.addConstr((small_use[(t,b)] == quicksum([bu[(a,b)]*s[(a,t-672*3)] for a in smallActIdx]) + 
                                                      quicksum([buo[(o,b)]*so[(o,t)] for o in smallOnceIdx])),name="buso4")
    
    for b in buildings:
        for t in week5:
            constr = m.addConstr((small_use[(t,b)] == quicksum([bu[(a,b)]*s[(a,t-672*4)] for a in smallActIdx]) + 
                                                      quicksum([buo[(o,b)]*so[(o,t)] for o in smallOnceIdx])),name="buso5")

    for b in buildings:
        for t in week1:
            constr = m.addConstr((large_use[(t,b)] == quicksum([bu[(a,b)]*s[(a,t)] for a in largeActIdx]) +
                                                      quicksum([buo[(o,b)]*so[(o,t)] for o in largeOnceIdx])),name="bulo1")
            
    for b in buildings:
        for t in week2:
            constr = m.addConstr((large_use[(t,b)] == quicksum([bu[(a,b)]*s[(a,t-672)] for a in largeActIdx]) +
                                                      quicksum([buo[(o,b)]*so[(o,t)] for o in largeOnceIdx])),name="bulo2")

    for b in buildings:
        for t in week3:
            constr = m.addConstr((large_use[(t,b)] == quicksum([bu[(a,b)]*s[(a,t-672*2)] for a in largeActIdx]) +
                                                      quicksum([buo[(o,b)]*so[(o,t)] for o in largeOnceIdx])),name="bulo3")

    for b in buildings:
        for t in week4:
            constr = m.addConstr((large_use[(t,b)] == quicksum([bu[(a,b)]*s[(a,t-672*3)] for a in largeActIdx]) +
                                                      quicksum([buo[(o,b)]*so[(o,t)] for o in largeOnceIdx])),name="bulo4")
        
    for b in buildings:
        for t in week5:
            constr = m.addConstr((large_use[(t,b)] == quicksum([bu[(a,b)]*s[(a,t-672*4)] for a in largeActIdx]) +
                                                      quicksum([buo[(o,b)]*so[(o,t)] for o in largeOnceIdx])),name="bulo5")            

    constr = m.addConstrs((small_use[(t,b)] <= building_small[b] for t in timeperiods for b in buildings), name="small_lim")
    constr = m.addConstrs((large_use[(t,b)] <= building_large[b] for t in timeperiods for b in buildings), name="large_lim")

    const = m.addConstrs((quicksum([bu[(a,b)] for b in buildings]) == ActRooms[a] for a in smallActIdx),name="smallrooms")
    const = m.addConstrs((quicksum([bu[(a,b)] for b in buildings]) == ActRooms[a] for a in largeActIdx),name="largerooms")

    #const = m.addConstrs((buoo[o,b] == buo[o,b] for o in Once for b in buildings), name="buoo_bin")

    const = m.addConstrs((quicksum([buo[(o,b)] for b in buildings]) == OnceRooms[o] for o in smallOnceIdx),name="smallroomso")
    const = m.addConstrs((quicksum([buo[(o,b)] for b in buildings]) == OnceRooms[o] for o in largeOnceIdx),name="largeroomso")
    
    constr = m.addConstrs((quicksum([s[(a,t)] for t in range(0,88)]) == 0 for a in Act),name="zero1") # to Mon 9am
    constr = m.addConstrs((quicksum([s[(a,t)] for t in range(120,184)]) == 0 for a in Act),name="zero2") # Mon 5pm-Tue 9am
    constr = m.addConstrs((quicksum([s[(a,t)] for t in range(216,280)]) == 0 for a in Act),name="zero3")# Tue 5pm-Wed 9am
    constr = m.addConstrs((quicksum([s[(a,t)] for t in range(312,376)]) == 0 for a in Act),name="zero4")# Wed 5pm-Thu 9am
    constr = m.addConstrs((quicksum([s[(a,t)] for t in range(408,472)]) == 0 for a in Act),name="zero5")# Thu 5pm-Fri 9am
    constr = m.addConstrs((quicksum([s[(a,t)] for t in range(504,T)]) == 0 for a in Act),name="zero6")# Fri 5pm-Mon 9am
    
    ## next constraints -- Once-off activities are ONLY in peak hours now 
    
    constr = m.addConstrs((quicksum([so[(o,t)] for t in range(0,88)]) == 0 for o in Once),name="zero1") # to Mon 9am
    constr = m.addConstrs((quicksum([so[(o,t)] for t in range(120,184)]) == 0 for o in Once),name="zero2") # Mon 5pm-Tue 9am
    constr = m.addConstrs((quicksum([so[(o,t)] for t in range(216,280)]) == 0 for o in Once),name="zero3")# Tue 5pm-Wed 9am
    constr = m.addConstrs((quicksum([so[(o,t)] for t in range(312,376)]) == 0 for o in Once),name="zero4")# Wed 5pm-Thu 9am
    constr = m.addConstrs((quicksum([so[(o,t)] for t in range(408,472)]) == 0 for o in Once),name="zero5")# Thu 5pm-Fri 9am
    constr = m.addConstrs((quicksum([so[(o,t)] for t in range(504,88+672)]) == 0 for o in Once),name="zero6")# Fri 5pm-Mon 9am
    
    #constr = m.addConstrs((quicksum([so[(a,t)] for t in range(0+672,88+672)]) == 0 for o in Once),name="zeroa1")
    constr = m.addConstrs((quicksum([so[(o,t)] for t in range(120+672,184+672)]) == 0 for o in Once),name="zeroa2")
    constr = m.addConstrs((quicksum([so[(o,t)] for t in range(216+672,280+672)]) == 0 for o in Once),name="zeroa3")
    constr = m.addConstrs((quicksum([so[(o,t)] for t in range(312+672,376+672)]) == 0 for o in Once),name="zeroa4")
    constr = m.addConstrs((quicksum([so[(o,t)] for t in range(408+672,472+672)]) == 0 for o in Once),name="zeroa5")
    constr = m.addConstrs((quicksum([so[(o,t)] for t in range(504+672,88+672*2)]) == 0 for o in Once),name="zeroa6")
    
    #constr = m.addConstrs((quicksum([so[(a,t)] for t in range(0+672*2,88+672*2)]) == 0 for o in Once),name="zerob1")
    constr = m.addConstrs((quicksum([so[(o,t)] for t in range(120+672*2,184+672*2)]) == 0 for o in Once),name="zerob2")
    constr = m.addConstrs((quicksum([so[(o,t)] for t in range(216+672*2,280+672*2)]) == 0 for o in Once),name="zerob3")
    constr = m.addConstrs((quicksum([so[(o,t)] for t in range(312+672*2,376+672*2)]) == 0 for o in Once),name="zerob4")
    constr = m.addConstrs((quicksum([so[(o,t)] for t in range(408+672*2,472+672*2)]) == 0 for o in Once),name="zerob5")
    constr = m.addConstrs((quicksum([so[(o,t)] for t in range(504+672*2,88+672*3)]) == 0 for o in Once),name="zerob6")
    
    #constr = m.addConstrs((quicksum([so[(a,t)] for t in range(0+672*3,88+672*3)]) == 0 for o in Once),name="zerob1")
    constr = m.addConstrs((quicksum([so[(o,t)] for t in range(120+672*3,184+672*3)]) == 0 for o in Once),name="zerob2")
    constr = m.addConstrs((quicksum([so[(o,t)] for t in range(216+672*3,280+672*3)]) == 0 for o in Once),name="zerob3")
    constr = m.addConstrs((quicksum([so[(o,t)] for t in range(312+672*3,376+672*3)]) == 0 for o in Once),name="zerob4")
    constr = m.addConstrs((quicksum([so[(o,t)] for t in range(408+672*3,472+672*3)]) == 0 for o in Once),name="zerob5")
    constr = m.addConstrs((quicksum([so[(o,t)] for t in range(504+672*3,88+672*4)]) == 0 for o in Once),name="zerob6")
    
    #constr = m.addConstrs((quicksum([so[(a,t)] for t in range(0+672*4,88+672*4)]) == 0 for o in Once),name="zero6")
    constr = m.addConstrs((quicksum([so[(o,t)] for t in range(120+672*4,184+672*4)]) == 0 for o in Once),name="zeroc2")
    #constr = m.addConstrs((quicksum([so[(a,t)] for t in range(216+672*4,184+672*4)]) == 0 for o in Once),name="zero6")
    
    ## but 184 + 672*4 = 2872 -- what happens to periods 2872 to 2880 -- Tue Dec 1 9am to 10.45am inclusive 
    
    #constr = m.addConstrs((recurload[t] == 0 for t in notweek1),name="zeror1")
    
    for a in Act:
        ro = pd.to_numeric(recur_data[a,6:])
        co = int(ro[0])
        for i in range(co):
            m.addConstr((z[a] - z[int(ro[i+1])] >= 65),name="prec")
            
    for o in Once:
        ro = pd.to_numeric(once_data[o,8:])
        co = int(ro[0])        
        ### next add constr         
        for i in range(co):
            #m.addConstr((math.floor(z[o]/96) - math.floor(z[int(ro[i+1])]/96) >= 1),name="prec1") ## has to be on a previous day -- let's say at least 16 hours ago
            #m.addConstr((onceIn[o]==1) >> (dayzo[o] - dayzo[int(ro[i+1])] >= 1),name="prec1")
            m.addConstr((dayzo[o] - dayzo[int(ro[i+1])] >= 1),name="prec1")

    constr = m.addConstrs((actLoad[(a,t)] == ActEnergy[a]*ActRooms[a]*s[(a,t)]) for a in Act for t in week1)
    constr = m.addConstrs((onceLoad[(o,t)] == OnceEnergy[o]*OnceRooms[o]*so[(o,t)]) for o in Once for t in weeks)#
 
    #################################     
        
    
#     sfile5 = sfile4.replace("mps","lp") # the given start point
#     m.write(sfile5)
    
#     m.update()
#     #m.read("flatsmall3_24.sol")
#     m.read(sol)
#     m.update()

#     for a in Act:
#         z[a].ub = z[a].Start
#         z[a].lb = z[a].Start
    
#     for a in Act:
#         for b in buildings:
#             bu[a,b].lb = bu[a,b].Start
#             bu[a,b].ub = bu[a,b].Start
            
    sfile = file.replace("phase2","p2sol\\flatonce_") # the given start point
    sfile2 = sfile.replace("_instance_","") # the given start point
    sfile3 = sfile2.replace("_","") # the given start point
    sfile4 = sfile3.replace("txt","mps") # the given start point
    m.write(sfile4)

    #m.Params.SolFiles="testfs4"
    #m.optimize()

In [ ]:
m.update()
sfile = file.replace("phase2","p2sol\\flatoncefix_") # the given start point
sfile2 = sfile.replace("_instance_","") # the given start point
sfile3 = sfile2.replace("_","") # the given start point
sfile4 = sfile3.replace("txt","mps") # the given start point
m.write(sfile4)

In [ ]:
m.optimize()